# Part 1: Importing Necessary Libraries

In [1]:
import os   
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import random

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import webbrowser
from json.decoder import JSONDecodeError

# pytest
import pytest
import unittest
from unittest.mock import patch, Mock

In [6]:
load_dotenv()
    
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")

# Part 2: Logging in With Authentication

In [9]:
# Create App With User Authentication

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
redirect_uri=REDIRECT_URI,
scope="user-library-read user-top-read"),
show_dialog=True)


results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], "-", track['name'])

Using 'localhost' as a redirect URI is being deprecated. Use a loopback IP address such as 127.0.0.1 to ensure your app remains functional.


0 Sports - Baby Baby
1 54 Ultra - Heaven Knows
2 Men I Trust - All My Candles
3 Phoebe Bridgers - Motion Sickness
4 Big Thief - Paul - 2023 Remaster
5 M.A.G.S. - Drugs
6 beabadoobee - Glue Song (feat. Clairo)
7 d4vd - Where'd It Go Wrong?
8 Beach House - Space Song
9 Suki Waterhouse - To Love
10 Selena Gomez - Ojos Tristes (with The Marías)
11 SZA - Sweet November
12 Omar Apollo - Unbothered
13 Dre'es - Warm
14 The Marías - Hold It Together
15 Memphis Cult - Ты бл*дь су*ка
16 Rviden - Vibração Potente
17 Seek - REI DO BRASIL
18 PHNKR - HARD DANÇA PHONK
19 $MXLE - MTG MISERÁVEL


In [18]:
# Fetch top 100 tracks
top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')

for idx, item in enumerate(top_tracks['items']):
    track_name = item['name']
    artist_name = item['artists'][0]['name']  
    print(f"{idx + 1}. {artist_name} - {track_name}")

1. The Marías - No One Noticed
2. Lunar Vacation - The Basement
3. The Marías - Care For You
4. Cannons - Fire for You
5. Faye Webster - Kingston
6. The Marías - Echo
7. The Marías - Run Your Mouth
8. Selena Gomez - Ojos Tristes (with The Marías)
9. Peach Pit - Techno Show
10. Clairo - Thank You
11. Doja Cat - URRRGE!!!!!!!!!! (feat. A$AP Rocky)
12. Paul Cherry - Hey Girl
13. Ravyn Lenae - Computer Luv (feat. Steve Lacy)
14. Tyler, The Creator - BEST INTEREST
15. Kid Travis - Strawberry Skies
16. Omar Apollo - Be Careful With Me
17. Djo - Roddy
18. Suki Waterhouse - OMG
19. Men I Trust - Numb
20. beabadoobee - If You Want To
21. Men I Trust - All Night
22. Still Woozy - Window
23. The Marías - I Don't Know You
24. Clairo - Amoeba
25. Faye Webster - Right Side of My Neck
26. Gregory Alan Isakov - Words
27. The Marías - Over the Moon
28. TOKYOSLEEP - PUMPING FUNK
29. The Marías - bop it up!
30. The Marías - Sienna
31. Steve Lacy - Give You the World
32. Megan Thee Stallion - Scary (feat.

In [35]:
# Get audio features for tracks
track_ids = [track['id'] for track in top_tracks['items']]
audio_features = sp.audio_features(track_ids)

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3siwsiaEoU4Kuuc9WKMUy5,1RXzYAbCsqYP6CcAkmPDM1,2CBtdZVcpSwaxOcLUi1AGo,4o0LyB69tylqDG6eTGhmig,5WbfFTuIldjL9x7W6y5l7R,3SH1vOTwgg5Ma1NhHaXvrA,19fKJrO9XdOf6Xla2QHecO,1DpC4L3JjsGRW7y6eTHaMj,0a0EYuSVqeVqV8kjN9bFYC,0mMlo76aVZHpuvoR6fjBSK,0mxMMetknt24PVQxAHwXAZ,2XZZqYiCSsKQPZT0TFef4L,2wsa9Re66HqlOkgb3DpnBX,3jHdKaLCkuNEkWcLVmQPCX,458EaCXcB9TRBoJEDVcdcr,4JOsabaJ1et99Ulb1E9GIE,5hS8s1Vf4CeSRBT1bTWa39,5vYjTN8d0DZ2SosbKSx5Nj,6Ijmj8Z0L31hCp5pLZnT5U,7I1kle4TNmkfednJDKo8GR,0XvoMOI25j4wODyk8nuANS,42Fm8hxhh8i0Zx0Si7XIxd,06cqIVC8kRAT02qfHQT65v,0HAqq2GcQKyi3s87GuN7jU,3cxZT78mZDyLsLPJKcTu3U,01Dc5vTMc9axpkvDUy0yiD,02wNtFxlr9dLmZXKT5TIF5,05d5Zzb4LwIY7FojboJQZi,06yhr4JeQjIcyDZwEwjGcm,0InIeZW4P6VO7dUGRM4AKH,0J119Oas2ox6JTTHUGZxHN,0QWauSvdEXExwUUDa4QCsj,0ZU7dTp8rNR1Bpc0jMcEAD,0ZUXj43fteJjwvGoLMntte,0fBFp8ItmAfAqPHZx8qQsh,0k2HFzw2iQENrU2oWtVJEb,0pJW1Xw3aY4Eh6k5iuBkfI,0trHOzAhNpGCsGBEu7dOJo,0zgDfVY84eFIzgTvK9KutN,13xVnZKZDEGbmXkzBFpJDD,15hJmqqEtAS

Spotify API error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3siwsiaEoU4Kuuc9WKMUy5,1RXzYAbCsqYP6CcAkmPDM1,2CBtdZVcpSwaxOcLUi1AGo,4o0LyB69tylqDG6eTGhmig,5WbfFTuIldjL9x7W6y5l7R,3SH1vOTwgg5Ma1NhHaXvrA,19fKJrO9XdOf6Xla2QHecO,1DpC4L3JjsGRW7y6eTHaMj,0a0EYuSVqeVqV8kjN9bFYC,0mMlo76aVZHpuvoR6fjBSK,0mxMMetknt24PVQxAHwXAZ,2XZZqYiCSsKQPZT0TFef4L,2wsa9Re66HqlOkgb3DpnBX,3jHdKaLCkuNEkWcLVmQPCX,458EaCXcB9TRBoJEDVcdcr,4JOsabaJ1et99Ulb1E9GIE,5hS8s1Vf4CeSRBT1bTWa39,5vYjTN8d0DZ2SosbKSx5Nj,6Ijmj8Z0L31hCp5pLZnT5U,7I1kle4TNmkfednJDKo8GR,0XvoMOI25j4wODyk8nuANS,42Fm8hxhh8i0Zx0Si7XIxd,06cqIVC8kRAT02qfHQT65v,0HAqq2GcQKyi3s87GuN7jU,3cxZT78mZDyLsLPJKcTu3U,01Dc5vTMc9axpkvDUy0yiD,02wNtFxlr9dLmZXKT5TIF5,05d5Zzb4LwIY7FojboJQZi,06yhr4JeQjIcyDZwEwjGcm,0InIeZW4P6VO7dUGRM4AKH,0J119Oas2ox6JTTHUGZxHN,0QWauSvdEXExwUUDa4QCsj,0ZU7dTp8rNR1Bpc0jMcEAD,0ZUXj43fteJjwvGoLMntte,0fBFp8ItmAfAqPHZx8qQsh,0k2HFzw2iQENrU2oWtVJEb,0pJW1Xw3aY4Eh6k5iuBkfI,0trHOzAhNpGCsGBEu7dOJo,0zgDfVY84eFIzgTvK9KutN,13xVnZKZ

In [ ]:
# Classify vibe based on audio features
def classify_vibe(features):
    energy = features['energy']
    valence = features['valence']
    acousticness = features['acousticness']
    danceability = features['danceability']
    instrumentalness = features['instrumentalness']
    speechiness = features['speechiness']
    
    # Vibe classification logic (very basic for now)
    if energy > 0.8 and danceability > 0.7 and valence > 0.7:
        return "Upbeat Party"
    elif valence > 0.6 and energy > 0.6:
        return "Happy/Uplifting"
    elif valence < 0.4 and energy < 0.5:
        return "Sad/Melancholic"
    elif acousticness > 0.7 and energy < 0.5:
        return "Chill/Acoustic"
    elif instrumentalness > 0.7:
        return "Cinematic/Instrumental"
    elif speechiness > 0.5:
        return "Rap/Spoken Word"
    elif danceability > 0.6 and energy > 0.6:
        return "Danceable/Groovy"
    else:
        return "Neutral/Mixed"

In [ ]:
# Combine track info with features and classified vibes
vibes_data = []
for i, features in enumerate(audio_features):
    if features is None:
        continue
    track = top_tracks['items'][i]
    vibe = classify_vibe(features)
    vibes_data.append({
        'track_name': track['name'],
        'artist_name': track['artists'][0]['name'],
        'vibe': vibe,
        'energy': features['energy'],
        'valence': features['valence'],
        'acousticness': features['acousticness'],
        'danceability': features['danceability'],
        'instrumentalness': features['instrumentalness'],
        'speechiness': features['speechiness']
    })

In [ ]:
# Plot vibes distribution
# Convert to DataFrame
vibes_df = pd.DataFrame(vibes_data)

# Plot vibes distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=vibes_df, x='vibe', order=vibes_df['vibe'].value_counts().index, palette='muted')
plt.xticks(rotation=45)
plt.title("Distribution of Vibes in Your Top Tracks")
plt.xlabel("Vibe Category")
plt.ylabel("Number of Tracks")
plt.tight_layout()
plt.show()